In [2]:
import pandas as pd
from sklearn.model_selection import train_test_split

import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
# suppress warnings
import warnings
warnings.filterwarnings('ignore')

# for visualization

C:\Users\86188\AppData\Local\Temp\ipykernel_23828\3735148473.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [3]:
fraud = pd.read_csv("https://cs307.org/lab-06/data/fraud.csv")
fraud_train, fraud_test = train_test_split(
    fraud,
    test_size=0.20,
    random_state=42,
    stratify=fraud["Fraud"],
)

# create X and y for train
X_train = fraud_train.drop("Fraud", axis=1)
y_train = fraud_train["Fraud"]

# create X and y for test
X_test = fraud_test.drop("Fraud", axis=1)
y_test = fraud_test["Fraud"]

In [31]:
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler, PolynomialFeatures
from sklearn.linear_model import Lasso
from sklearn.model_selection import GridSearchCV
import numpy as np
from sklearn.metrics import precision_score, recall_score, f1_score, make_scorer


pipeline = Pipeline([
    ('scaler', StandardScaler()),
    ('lasso', Lasso(max_iter=10000))
])

param_grid = {
    'lasso__alpha': np.logspace(-10, 1, 100),
}
precision_scorer = make_scorer(precision_score)
recall_scorer = make_scorer(recall_score)
#f1_scorer = make_scorer(f1_score)


grid_search = GridSearchCV(
    pipeline,
    param_grid,
    cv=5,
    scoring={'recall': recall_scorer,
             'precision': precision_scorer},
    refit='recall',  # 指定用于重新训练最佳模型的评分函数
    verbose=1,
    return_train_score=True
)

# 在训练数据上执行网格搜索
grid_search.fit(X_train, y_train)
print("Best parameters: ", grid_search.best_params_)
y_pred = grid_search.predict(X_test)

Fitting 5 folds for each of 100 candidates, totalling 500 fits
Best parameters:  {'lasso__alpha': 1e-10}


In [32]:
for i in range(0, len(y_pred)):
    if y_pred[i] > 0.5:
        y_pred[i] = 1
    else:
        y_pred[i] = 0
y_pred

array([0., 0., 0., ..., 0., 0., 0.])

In [33]:
y_test

37496    0
36852    0
66528    0
49536    0
31000    0
        ..
35125    0
55376    0
20282    0
67620    0
14280    0
Name: Fraud, Length: 13569, dtype: int64

In [34]:
# 计算精确度
precision = precision_score(y_test, y_pred)
print("Precision: ", precision)

Precision:  0.975609756097561


In [35]:
# 计算召回率
recall = recall_score(y_test, y_pred)
print("Recall: ", recall)

Recall:  0.5063291139240507
